In [3]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from threading import Thread
import os
os.environ['CUDA_VISIBLE_DEVICES']='0,1'

In [12]:
# https://huggingface.co/blog/llama2#using-transformers


tiny_llama = "/mnt/xue.w/models/hub/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/snapshots/77e23968eed12d195bd46c519aa679cc22a27ddc"
llama_7b_hf_chat = "/mnt/xue.w/models/hub/models--meta-llama--Llama-2-7b-chat-hf/snapshots/c1b0db933684edbfe29a06fa47eb19cc48025e93"
# code_llama_70 = '/mnt/xue.w/models/hub/models--codellama--CodeLlama-70b-hf/snapshots/4570a4edc524fb9f20f605b417bb43828fa5997a'

miqu_70b ='/mnt/xue.w/models/hub/models--miqudev--miqu-1-70b/models--miqudev--miqu-1-70b/snapshots/82f0daa6767263aa5990dea54dbb13e94d096de7'
Mixtral_8x7b_instruct ='/mnt/xue.w/models/hub/models--mistralai--Mixtral-8x7B-Instruct-v0.1/snapshots/125c431e2ff41a156b9f9076f744d2f35dd6e67a'
model_pth = Mixtral_8x7b_instruct




# Loading the tokenizer and model from Hugging Face's model hub.
tokenizer = AutoTokenizer.from_pretrained(model_pth,torch_dtype=torch.float16,device_map="auto",load_in_4bit=False,load_in_8bit=False)
model = AutoModelForCausalLM.from_pretrained(model_pth,torch_dtype=torch.float16,device_map="auto",load_in_4bit=False,load_in_8bit=False)

# using CUDA for an optimal experience
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [2]  # IDs of tokens where the generation should stop.
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:  # Checking if the last generated token is a stop token.
                return True
        return False


# Function to generate model predictions.
def predict(message,history = []):
    history_transformer_format = history + [[message, ""]]
    stop = StopOnTokens()

    # Formatting the input for the model.
    messages = "</s>".join(["</s>".join(["\n<|user|>:" + item[0], "\n<|assistant|>:" + item[1]])
                        for item in history_transformer_format])
    model_inputs = tokenizer([messages], return_tensors="pt")
    # .to(device)
    streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        model_inputs,
        streamer=streamer,
        max_new_tokens=1024,
        do_sample=True,
        top_p=0.95,
        top_k=50,
        temperature=0.7,
        num_beams=1,
        stopping_criteria=StoppingCriteriaList([stop])
    )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()  # Starting the generation in a separate thread.
    partial_message = ""
    for new_token in streamer:
        partial_message += new_token
        if '</s>' in partial_message:  # Breaking the loop if the stop token is generated.
            break
    yield partial_message
    # return partial_message


OSError: /mnt/xue.w/models/hub/models--miqudev--miqu-1-70b/models--miqudev--miqu-1-70b/snapshots/82f0daa6767263aa5990dea54dbb13e94d096de7 does not appear to have a file named config.json. Checkout 'https://huggingface.co//mnt/xue.w/models/hub/models--miqudev--miqu-1-70b/models--miqudev--miqu-1-70b/snapshots/82f0daa6767263aa5990dea54dbb13e94d096de7/None' for available files.

In [9]:
a0 = 'What is the difference between How old are you and how are you?'
for i in predict(a0):
    print(i)
# a0 = 'What is the difference between How old are you and how are you?'
# a = predict(a0)
# print(a)
# print('---'*20)
# a0 = 'If 1 = 6, 2 = 5, 3=4, what should 5 = ?'
# a = predict(a0)
# print(a)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"How old are you?" is a question that asks about a person's age, while "How are you?" is a question that asks about a person's current state of being, often referring to their health or mood.


In [7]:
a0 = 'If 1 = 6, 2 = 5, 3=4, what should 5 = ?'
for i in predict(a0):
    print(i)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




Given 
Given the 
Given the 
Given the pattern, 
Given the pattern, if 
Given the pattern, if 
Given the pattern, if 1 
Given the pattern, if 1 = 
Given the pattern, if 1 = 
Given the pattern, if 1 = 
Given the pattern, if 1 = 6, 
Given the pattern, if 1 = 6, 
Given the pattern, if 1 = 6, 2 
Given the pattern, if 1 = 6, 2 = 
Given the pattern, if 1 = 6, 2 = 
Given the pattern, if 1 = 6, 2 = 
Given the pattern, if 1 = 6, 2 = 5, 
Given the pattern, if 1 = 6, 2 = 5, and 
Given the pattern, if 1 = 6, 2 = 5, and 
Given the pattern, if 1 = 6, 2 = 5, and 3 
Given the pattern, if 1 = 6, 2 = 5, and 3 = 
Given the pattern, if 1 = 6, 2 = 5, and 3 = 
Given the pattern, if 1 = 6, 2 = 5, and 3 = 
Given the pattern, if 1 = 6, 2 = 5, and 3 = 4, 
Given the pattern, if 1 = 6, 2 = 5, and 3 = 4, then 
Given the pattern, if 1 = 6, 2 = 5, and 3 = 4, then 
Given the pattern, if 1 = 6, 2 = 5, and 3 = 4, then 5 
Given the pattern, if 1 = 6, 2 = 5, and 3 = 4, then 5 would 
Given the pattern, if 1 = 6, 2 = 5, 